In [1]:
import numpy as np
import pandas as pd

In [2]:
DATA_SOURCE = "/Users/mac/Desktop/DSTI_Project/Machine_Learning/Project 2/archive/"

In [3]:
pd.options.display.float_format = '{:,.0f}'.format

In [4]:
pd.options.display.float_format = "{:.2f}".format

In [5]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", 1000)

In [6]:
prior = pd.read_csv(DATA_SOURCE + "order_products__prior.csv")

In [7]:
train = pd.read_csv(DATA_SOURCE + "order_products__train.csv")

In [8]:
orders = pd.read_csv(DATA_SOURCE + "orders.csv")

In [9]:
orders_prior = orders[orders["eval_set"] == "prior"].copy()

In [10]:
orders_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.00
2,473747,1,prior,3,3,12,21.00
3,2254736,1,prior,4,4,7,29.00
4,431534,1,prior,5,4,15,28.00


In [11]:
orders_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.00
2,473747,1,prior,3,3,12,21.00
3,2254736,1,prior,4,4,7,29.00
4,431534,1,prior,5,4,15,28.00


In [12]:
df_merged_prior_order_prior = pd.merge(prior, orders_prior, on = "order_id", how = "left")

In [13]:
df_merged_prior_order_prior = df_merged_prior_order_prior.sort_values(by = ["user_id", "order_id"])

In [14]:
avg_reordered = (
    df_merged_prior_order_prior
    .groupby(["user_id"])["reordered"]
    .mean()
    .reset_index()
    .rename(columns = {"reordered":"avg_reordered"})
)

In [15]:
avg_reordered.head()

,user_id,avg_reordered
0,1,0.69
1,2,0.48
2,3,0.62
3,4,0.06
4,5,0.38


In [16]:
avg_reordered_by_product = (
    df_merged_prior_order_prior
    .groupby(["user_id", "product_id"])["reordered"]
    .mean()
    .reset_index(name="avg_reordered_by_product")
)

In [17]:
avg_reordered_by_product.head()

,user_id,product_id,avg_reordered_by_product
0,1,196,0.90
1,1,10258,0.89
2,1,10326,0.00
3,1,12427,0.90
4,1,13032,0.67


In [18]:
train = pd.merge(train, orders, on = "order_id", how = "left")

In [19]:
train.describe()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,1384617.00,1384617.00,1384617.00,1384617.00,1384617.00,1384617.00,1384617.00,1384617.00,1384617.00
mean,1706297.62,25556.24,8.76,0.60,103112.78,17.09,2.70,13.58,17.07
std,989732.65,14121.27,7.42,0.49,59487.15,16.61,2.17,4.24,10.43
min,1.00,1.00,1.00,0.00,1.00,4.00,0.00,0.00,0.00
25%,843370.00,13380.00,3.00,0.00,51732.00,6.00,1.00,10.00,7.00
50%,1701880.00,25298.00,7.00,1.00,102933.00,11.00,3.00,14.00,15.00
75%,2568023.00,37940.00,12.00,1.00,154959.00,21.00,5.00,17.00,30.00
max,3421070.00,49688.00,80.00,1.00,206209.00,100.00,6.00,23.00,30.00


In [20]:
train = train.drop(columns=["order_id", "eval_set", "order_number", "order_hour_of_day", "add_to_cart_order"])

In [21]:
train.head()

,product_id,reordered,user_id,order_dow,days_since_prior_order
0,49302,1,112108,4,9.00
1,11109,1,112108,4,9.00
2,10246,0,112108,4,9.00
3,49683,0,112108,4,9.00
4,43633,1,112108,4,9.00


In [22]:
train = train.merge(avg_reordered_by_product, on = ["user_id", "product_id"], how="left")

In [23]:
train = train.merge(avg_reordered, on = "user_id", how="left")

In [24]:
train = train.sort_values(by = ["avg_reordered"], ascending=False)

In [25]:
train.isna().sum()

product_id                       0
reordered                        0
user_id                          0
order_dow                        0
days_since_prior_order           0
avg_reordered_by_product    555793
avg_reordered                    0
dtype: int64

In [26]:
train["avg_reordered_by_product"] = train["avg_reordered_by_product"].fillna(0)

In [27]:
train.isna().sum()

product_id                  0
reordered                   0
user_id                     0
order_dow                   0
days_since_prior_order      0
avg_reordered_by_product    0
avg_reordered               0
dtype: int64

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
df_train, df_test = train_test_split(train,test_size = 0.2)

In [30]:
len(df_train)

1107693

In [31]:
df_train[["product_id", "user_id", "reordered", "avg_reordered", "avg_reordered_by_product"]].head(60)

,product_id,user_id,reordered,avg_reordered,avg_reordered_by_product
2595,47766,136194,1,0.24,0.00
862046,42450,110482,0,0.13,0.00
781047,20014,166360,0,0.56,0.00
825584,21936,30113,1,0.71,0.83
718189,35851,51184,1,0.47,0.86
1200667,34050,182835,1,0.23,0.50
1068383,10895,66495,1,0.61,0.50
364170,46906,17838,0,0.48,0.00
1180937,38456,117883,0,0.47,0.00
352407,35383,15313,1,0.55,0.00


In [32]:
df_train.describe()

,product_id,reordered,user_id,order_dow,days_since_prior_order,avg_reordered_by_product,avg_reordered
count,1107693.00,1107693.00,1107693.00,1107693.00,1107693.00,1107693.00,1107693.00
mean,25548.67,0.60,103090.22,2.70,17.07,0.31,0.45
std,14127.04,0.49,59504.76,2.17,10.43,0.38,0.21
min,1.00,0.00,1.00,0.00,0.00,0.00,0.00
25%,13340.00,0.00,51690.00,1.00,7.00,0.00,0.29
50%,25288.00,1.00,102910.00,3.00,15.00,0.00,0.45
75%,37947.00,1.00,154967.00,5.00,30.00,0.67,0.61
max,49688.00,1.00,206209.00,6.00,30.00,0.99,0.99


In [33]:
len(df_test)

276924

In [34]:
X_train = df_train.loc[:,['product_id', 
                          'user_id', 
                          'avg_reordered_by_product']].values
y_train = df_train.reordered.values

In [35]:
# get the values of the columns for the test data
X_test = df_test.loc[:,['product_id', 
                          'user_id', 
                          'avg_reordered_by_product']].values
y_test = df_test.reordered.values

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
lr_model = LogisticRegression(random_state=0,max_iter=2000)

In [38]:
lr_model.fit(X=X_train, y=y_train)

LogisticRegression(max_iter=2000, random_state=0)

In [39]:
y_test_predicted = lr_model.predict(X_test)

In [40]:
(y_test_predicted == y_test).sum()/len(y_test)

0.8267611330184454